In [1]:
import numpy as np
import pandas as pd
import os.path as op
import os


inlist = lambda x, y: [x_ in y for x_ in x]

### Organize Dset100x1x1x20

In [2]:
df = pd.read_hdf('../data/impactofinstability_connect+feature_dset100x1x1x20_both.h5')
df = df.reset_index()

# Very incomplete sub: A00059344
inc = ['A00059344']
print(len(df))
for __ in inc:
    df = df[df.subject != __]
print(len(df))

# Broken graph subs
N_rois = 83
dfl = [row
       for idx, row in df.iterrows()
       if row['graph'].shape == (N_rois, N_rois)]
df = pd.DataFrame(dfl)
print(len(df))

# Drop pipelines and perturbation methods not of interest
df = df[df["pipeline"] == "prob"]
print(len(df))
df = df[df["instrumentation"] == "Data"]
print(len(df))
df.reset_index(drop=True, inplace=True)

# Drop some columns (will re-add relevant ones later)
df.drop(labels=['bmi', 'pulse', 'age', 'sex', 'index',
                'pipeline', 'instrumentation'], axis=1, inplace=True)

subs = list(set(df.subject))

8353
8309
8225
4111
2057


In [3]:
# Load DF
df_pheno = pd.read_csv('../data/phenotypic.csv')

# Deal with minor reorg details
df_pheno["session"] = [_.split(',')[1] for _ in df_pheno['id'].values]
df_pheno["id"] = [_.split(',')[0] for _ in df_pheno['id'].values]

keep = inlist(df_pheno['id'], subs)
df_pheno = df_pheno[keep]

for col in ['sex', 'vo2max', 'bmi', 'cholesterol']:
    tmp = [str(_).split()[0]
           for _ in df_pheno[col].values]
    df_pheno[col] = [np.nan if np.isnan(float(_))
                     else _
                     for _ in tmp]

# Prune for completeness
df_pheno.reset_index(drop=True, inplace=True)

In [4]:
for cov in ['age', 'sex', 'bmi', 'cholesterol', 'vo2max']:
    df[cov] = np.nan

for sub in subs:
    for cov in ['age', 'sex', 'bmi', 'cholesterol', 'vo2max']:
        df.loc[df['subject'] == sub, cov] = df_pheno[df_pheno['id'] == sub][cov].values[0]

In [6]:
df.to_hdf('../data/aggregation_connect+feature+demo_dset100x1x1x20.h5', 'data')

/Users/greg/code/env/stability/lib/python3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['subject', 'simulation', 'graph', 'degree', 'weight', 'ccoeff',
       'betweenness', 'plength', 'sex', 'bmi', 'cholesterol', 'vo2max'],
      dtype='object')]

  encoding=encoding,


### Organize Dset25x2x2x20

In [15]:
df = pd.read_hdf('../data/impactofinstability_connect_dset25x2x2x20_inputs.h5')
df = df.reset_index(drop=True)

# Drop pipeline not of interest
df = df[df["pipeline"] == "prob"]
print(len(df))
df.reset_index(drop=True, inplace=True)

# Drop some columns (will re-add relevant ones later)
df.drop(labels=['bmi', 'pulse', 'age', 'sex', 'pipeline'], axis=1, inplace=True)

subs = list(set(df.subject))

2100


In [19]:
df_pheno = pd.read_csv('../data/phenotypic.csv')

# Deal with minor reorg details
df_pheno["session"] = [_.split(',')[1] for _ in df_pheno['id'].values]
df_pheno["id"] = [_.split(',')[0] for _ in df_pheno['id'].values]

keep = inlist(df_pheno['id'], subs)
df_pheno = df_pheno[keep]

for col in ['sex', 'vo2max', 'bmi', 'cholesterol']:
    tmp = [str(_).split()[0]
           for _ in df_pheno[col].values]
    df_pheno[col] = [np.nan if np.isnan(float(_))
                     else _
                     for _ in tmp]

# Prune for completeness
df_pheno.reset_index(drop=True, inplace=True)

In [24]:
for cov in ['age', 'sex', 'bmi', 'cholesterol', 'vo2max']:
    df[cov] = np.nan

for sub in subs:
    for cov in ['age', 'sex', 'bmi', 'cholesterol', 'vo2max']:
        df.loc[df['subject'] == sub, cov] = df_pheno[df_pheno['id'] == sub][cov].values[0]

In [27]:
df.to_hdf('../data/aggregation_connect+demo_dset25x2x2x20.h5', 'data')

/Users/greg/code/env/stability/lib/python3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['directions', 'graph', 'session', 'simulation', 'subject', 'sex', 'bmi',
       'cholesterol', 'vo2max'],
      dtype='object')]

  encoding=encoding,
